In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [4]:
# Opening JSON file
f = open('MMHS150K_GT.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
text = []
labels = []
# Closing file
f.close()
# Iterating through the json
# list
for i in data:
    text.append(data[i]["tweet_text"].split("https://")[0])
    labels.append(data[i]["labels_str"][0])

In [7]:
training, testing, y_train, y_test = train_test_split(text, labels, test_size=0.2)

# Load GloVe word vectors
def load_glove_vectors(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Global Embeddings File:
glove_file_path = 'glove.6B.50d.txt'
glove_embeddings_index = load_glove_vectors(glove_file_path)

# Function to get average GloVe vector for a sentence
def get_average_glove_vector(sentence, embeddings_index, embedding_dim):
    words = sentence.split()
    vectors = [embeddings_index.get(word, np.zeros((embedding_dim,))) for word in words]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros((embedding_dim,))

# Map sentences to GloVe embeddings
embedding_dim = len(next(iter(glove_embeddings_index.values())))
x_training = np.array([get_average_glove_vector(sentence, glove_embeddings_index, embedding_dim) for sentence in training])
x_testing = np.array([get_average_glove_vector(sentence, glove_embeddings_index, embedding_dim) for sentence in testing])

x_training_shifted = x_training - np.min(x_training)
x_testing_shifted = x_testing - np.min(x_testing)

NBmodel = MultinomialNB()
NBmodel.fit(x_training_shifted, y_train)

print("Training accuracy: " + str(NBmodel.score(x_training_shifted, y_train)))
print("Testing accuracy: " + str(NBmodel.score(x_testing_shifted, y_test)))

Training accuracy: 0.6970832151379132
Testing accuracy: 0.7023861171366594
